In [1]:
import pandas as pd

In [6]:
df = pd.read_csv('D:/ESCRITORIO/Github/Road to data science I/Database/IGN games from best to worst.csv')
df.head(3)

,title,score,score_phrase,platform,genre,release_year,release_month,release_day
0,Checkered Flag,10.0,Masterpiece,Lynx,Racing,1999,7,6
1,Chrono Trigger,10.0,Masterpiece,Wii,"Action, RPG",2011,5,25
2,Dragon Warrior III,10.0,Masterpiece,Game Boy Color,RPG,2001,7,20


I 'll asume that the name of the game (title) isn't relevant for the final score, so I 'll drop this column. The release year should be important due to the genre of the game. There were some year where a certain genres were the most played or significant. However, the release month and day could be ignored (It is considered that these information in least impactful than the year of release).

In [16]:
df1 = df.drop(['title','release_month','release_day'], axis='columns')
df1.head(3)

,score,score_phrase,platform,genre,release_year
0,10.0,Masterpiece,Lynx,Racing,1999
1,10.0,Masterpiece,Wii,"Action, RPG",2011
2,10.0,Masterpiece,Game Boy Color,RPG,2001


In [17]:
df1.shape

(18625, 5)

In [22]:
df1.isna().sum()

score            0
score_phrase     0
platform         0
genre           36
release_year     0
dtype: int64

There are 36 null cells in the database related to the genre column. Because this quantity is quite low in comparison with the total no. of raws, I 'll drop them.

In [26]:
df2 = df1[df1.genre.isnull() == False]
df2.isna().sum()

score           0
score_phrase    0
platform        0
genre           0
release_year    0
dtype: int64

In [47]:
df2.genre.value_counts()

Action                      3797
Sports                      1916
Shooter                     1610
Racing                      1228
Adventure                   1175
                            ... 
RPG, Action                    1
Strategy, Simulation           1
Compilation, Compilation       1
Action, Editor                 1
Action, Strategy               1
Name: genre, Length: 112, dtype: int64

In [49]:
len(df2.genre.unique())

112

I 'll do one hot encoding to handle with the genre column.

In [53]:
ohe = pd.get_dummies(df2.genre)
ohe.head(3)

,Action,"Action, Adventure","Action, Compilation","Action, Editor","Action, Platformer","Action, Puzzle","Action, RPG","Action, Simulation","Action, Strategy","Adult, Card",...,"Sports, Racing","Sports, Simulation",Strategy,"Strategy, Compilation","Strategy, RPG","Strategy, Simulation",Trivia,Virtual Pet,Wrestling,"Wrestling, Simulation"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
df3 = pd.concat([df2,ohe],axis='columns')
df3.head()

,score,score_phrase,platform,genre,release_year,Action,"Action, Adventure","Action, Compilation","Action, Editor","Action, Platformer",...,"Sports, Racing","Sports, Simulation",Strategy,"Strategy, Compilation","Strategy, RPG","Strategy, Simulation",Trivia,Virtual Pet,Wrestling,"Wrestling, Simulation"
0,10.0,Masterpiece,Lynx,Racing,1999,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10.0,Masterpiece,Wii,"Action, RPG",2011,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10.0,Masterpiece,Game Boy Color,RPG,2001,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10.0,Masterpiece,Xbox 360,"Action, Adventure",2008,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10.0,Masterpiece,PlayStation 3,"Action, Adventure",2008,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


I 'll drop a column from the new ones incorporated to avoid the Dummy Variable Trap.

In [71]:
df4 = df3.drop('Action, Adventure', axis='columns')
df4.head(3)

,score,score_phrase,platform,genre,release_year,Action,"Action, Compilation","Action, Editor","Action, Platformer","Action, Puzzle",...,"Sports, Racing","Sports, Simulation",Strategy,"Strategy, Compilation","Strategy, RPG","Strategy, Simulation",Trivia,Virtual Pet,Wrestling,"Wrestling, Simulation"
0,10.0,Masterpiece,Lynx,Racing,1999,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10.0,Masterpiece,Wii,"Action, RPG",2011,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10.0,Masterpiece,Game Boy Color,RPG,2001,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


For the case of the score_phrase column, I 'll map the different values to numeric values through a dictionary.

In [72]:
df4.score_phrase.unique()

array(['Masterpiece', 'Amazing', 'Great', 'Good', 'Okay', 'Mediocre',
       'Bad', 'Awful', 'Painful', 'Unbearable', 'Disaster'], dtype=object)

In [73]:
df4.score_phrase.value_counts()

Great          4765
Good           4725
Okay           2940
Mediocre       1957
Amazing        1803
Bad            1268
Awful           662
Painful         339
Unbearable       72
Masterpiece      55
Disaster          3
Name: score_phrase, dtype: int64

In [79]:
dictionary = {}
cont = 0
for key in df4.score_phrase.unique():
    dictionary[key] = cont
    cont += 1

In [84]:
df4['score_phrase_num'] = df4.score_phrase.map(dictionary)
df4.head(3)

,score,score_phrase,platform,genre,release_year,Action,"Action, Compilation","Action, Editor","Action, Platformer","Action, Puzzle",...,"Sports, Simulation",Strategy,"Strategy, Compilation","Strategy, RPG","Strategy, Simulation",Trivia,Virtual Pet,Wrestling,"Wrestling, Simulation",score_phrase_num
0,10.0,Masterpiece,Lynx,Racing,1999,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10.0,Masterpiece,Wii,"Action, RPG",2011,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10.0,Masterpiece,Game Boy Color,RPG,2001,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


I 'll drop the genre and score_phrase columns just to have the dataframe more cleaned.

In [86]:
df5 = df4.drop(['score_phrase','genre'], axis='columns')
df5.head(3)

,score,platform,release_year,Action,"Action, Compilation","Action, Editor","Action, Platformer","Action, Puzzle","Action, RPG","Action, Simulation",...,"Sports, Simulation",Strategy,"Strategy, Compilation","Strategy, RPG","Strategy, Simulation",Trivia,Virtual Pet,Wrestling,"Wrestling, Simulation",score_phrase_num
0,10.0,Lynx,1999,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10.0,Wii,2011,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,10.0,Game Boy Color,2001,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
df5.platform.value_counts()

PC                      3367
PlayStation 2           1684
Xbox 360                1631
Wii                     1362
PlayStation 3           1355
Nintendo DS             1044
PlayStation              952
Wireless                 906
iPhone                   828
Xbox                     821
PlayStation Portable     632
Game Boy Advance         623
GameCube                 509
Game Boy Color           356
Nintendo 64              302
Dreamcast                286
PlayStation 4            277
Nintendo DSi             252
Nintendo 3DS             222
Xbox One                 208
PlayStation Vita         155
Wii U                    114
iPad                      99
Lynx                      82
Macintosh                 81
Genesis                   58
NES                       49
TurboGrafx-16             40
Android                   39
Super NES                 33
NeoGeo Pocket Color       31
N-Gage                    30
Game Boy                  22
iPod                      17
Sega 32X      

One hot encoding again

In [88]:
ohe = pd.get_dummies(df5.platform)
ohe.head(3)

,Android,Arcade,Atari 2600,Atari 5200,Commodore 64/128,DVD / HD Video Game,Dreamcast,Dreamcast VMU,Game Boy,Game Boy Advance,...,Windows Surface,Wireless,WonderSwan,WonderSwan Color,Xbox,Xbox 360,Xbox One,iPad,iPhone,iPod
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
df6 = pd.concat([df5,ohe],axis='columns')
df6.head(3)

,score,platform,release_year,Action,"Action, Compilation","Action, Editor","Action, Platformer","Action, Puzzle","Action, RPG","Action, Simulation",...,Windows Surface,Wireless,WonderSwan,WonderSwan Color,Xbox,Xbox 360,Xbox One,iPad,iPhone,iPod
0,10.0,Lynx,1999,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10.0,Wii,2011,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,10.0,Game Boy Color,2001,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


To avoid Dummy variable trap, I 'll drop SteamOS column (choosed randomly). Also to finally obtained the definitve numeric dataframe, I 'll drop platform column

In [93]:
df7 = df6.drop(['platform','SteamOS'], axis='columns')
df7.head(3)

,score,release_year,Action,"Action, Compilation","Action, Editor","Action, Platformer","Action, Puzzle","Action, RPG","Action, Simulation","Action, Strategy",...,Windows Surface,Wireless,WonderSwan,WonderSwan Color,Xbox,Xbox 360,Xbox One,iPad,iPhone,iPod
0,10.0,1999,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10.0,2011,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10.0,2001,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
from sklearn.model_selection import train_test_split

X = df7.drop('score', axis='columns')
y = df7.score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1996)

In [96]:
X_train.shape

(14871, 171)

In [97]:
X_test.shape

(3718, 171)

In [98]:
y_train.shape

(14871,)

In [100]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [101]:
lr_model.score(X_train, y_train)

0.9702843383063906

In [102]:
lr_model.score(X_test, y_test)

0.9706490192597131

In [127]:
lr_model.predict(X_test[:5])

array([6.34828045, 5.26606112, 8.49408932, 8.34186981, 5.42689463])

In [126]:
y_test[:5]

C:\Users\RAMOS\AppData\Local\Temp\ipykernel_21660\1754177261.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_test[:5]


11477    6.9
15973    5.0
4157     8.3
4448     8.2
15473    5.1
Name: score, dtype: float64

In [103]:
from sklearn.pipeline import Pipeline

In [144]:
X_train[:1]

,release_year,Action,"Action, Compilation","Action, Editor","Action, Platformer","Action, Puzzle","Action, RPG","Action, Simulation","Action, Strategy","Adult, Card",...,Windows Surface,Wireless,WonderSwan,WonderSwan Color,Xbox,Xbox 360,Xbox One,iPad,iPhone,iPod
7841,2004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [201]:
X_train.iloc[0]

release_year           2004
Action                    0
Action, Compilation       0
Action, Editor            0
Action, Platformer        0
                       ... 
Xbox 360                  0
Xbox One                  0
iPad                      0
iPhone                    0
iPod                      0
Name: 7841, Length: 171, dtype: int64

In [132]:
df_test = df
df_test.head()

,title,score,score_phrase,platform,genre,release_year,release_month,release_day
0,Checkered Flag,10.0,Masterpiece,Lynx,Racing,1999,7,6
1,Chrono Trigger,10.0,Masterpiece,Wii,"Action, RPG",2011,5,25
2,Dragon Warrior III,10.0,Masterpiece,Game Boy Color,RPG,2001,7,20
3,Grand Theft Auto IV,10.0,Masterpiece,Xbox 360,"Action, Adventure",2008,4,25
4,Grand Theft Auto IV,10.0,Masterpiece,PlayStation 3,"Action, Adventure",2008,4,25


In [136]:
X_row_raw = df[:0]
X_row_raw

,title,score,score_phrase,platform,genre,release_year,release_month,release_day


In [274]:
#raw_variable = {'score_phrase':}

def preprocess(X_row_raw):
    
    df_t = pd.read_csv('D:/ESCRITORIO/Github/Road to data science I/Database/IGN games from best to worst.csv')
    df_t = df_t.drop('score',axis='columns')
    
    X = pd.DataFrame([X_row_raw], columns=['title','score_phrase','platform','genre','release_year','release_month','release_day'])
    
    X = pd.concat([X,df_t])
    
    X = X.drop(['title','release_month','release_day'], axis='columns')
    
    ohe = pd.get_dummies(X.genre)
    
    X = pd.concat([X,ohe],axis='columns')
    
    X = X.drop('Action, Adventure', axis='columns')
    
    dictionary = {}
    cont = 0
    for key in df.score_phrase.unique():
        dictionary[key] = cont
        cont += 1
    
    X['score_phrase_num'] = X.score_phrase.map(dictionary)
    
    X = X.drop(['score_phrase','genre'], axis='columns')
    
    ohe = pd.get_dummies(X.platform)
    
    X = pd.concat([X,ohe],axis='columns')
    
    X = X.drop(['platform','SteamOS'], axis='columns')
    
    return X.iloc[:1]
    

    

    
    
    

In [276]:
X = ['Checkered Flag','Masterpiece','Lynx','Racing',1999,7,6]
preprocess(X)


,release_year,Action,"Action, Compilation","Action, Editor","Action, Platformer","Action, Puzzle","Action, RPG","Action, Simulation","Action, Strategy","Adult, Card",...,Windows Surface,Wireless,WonderSwan,WonderSwan Color,Xbox,Xbox 360,Xbox One,iPad,iPhone,iPod
0,1999,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [333]:
def prediction_func(X):
    X = preprocess(X)
    y_pred = lr_model.predict(X)
    for i in range(len(y_pred)):
        if y_pred[i] > 10:
            y_pred[i] = 10
    
    return y_pred

In [338]:
X = ['Checkered Flag','Masterpiece','Lynx','Racing',1999,7,6]
prediction_func(X)


array([9.96112536])

In [339]:
X = ['Checkered Flag','Disaster','Game Boy','Racing',1999,7,6]
prediction_func(X)

array([0.15124689])